In [1]:
%pip install -q  pyiceberg
!pip install -q duckdb
%pip install -q getdaft

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 9, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



**<mark>Import</mark>**

In [2]:
from   pyiceberg.catalog.sql import SqlCatalog
from   pyiceberg.catalog import load_catalog
import duckdb
import re ,shutil,glob
from   urllib.request import urlopen
import os
import requests
import multiprocessing
from   shutil import unpack_archive
import daft

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 11, Finished, Available, Finished)

In [3]:
catalog_type ='local' ## local or rest
warehouse_path = "/lakehouse/default/Files/iceberg"
os.makedirs(warehouse_path, exist_ok=True)
db_path = f"{warehouse_path}/pyiceberg_catalog.db"

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 12, Finished, Available, Finished)

**<mark>Connect to the Catalog</mark>**

In [4]:
def connect_catalog(type):
  if type =='local':
    catalog = SqlCatalog(
        "default",
        **{
            "uri": "sqlite:///" + db_path,
            "warehouse": f"file://{warehouse_path}",
        },
    )
  else:
    duckdb.sql(f""" CREATE or replace SECRET onelake (TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{notebookutils.credentials.getToken('storage')}') """)
    kv =duckdb.sql(""" select trim(split_part("[myvars]",' : ', 1) ) as var, trim(split_part("[myvars]",' : ', 2)) as values from
    read_csv('abfss://iceberg@onelake.dfs.fabric.microsoft.com/iceberg.Lakehouse/Files/variable.ini')
            """)
    kv_values=duckdb.sql( """ from kv where var
     in ('polaris_key')
      """).fetchall()
    for key, value in kv_values:
        globals()[key] = value
        print(key)
    catalog = load_catalog(
            'default',
            uri='https://jltvfet-polaris.snowflakecomputing.com/polaris/api/catalog',
            warehouse='dwh',
            scope = 'PRINCIPAL_ROLE:data_engineer' ,
            credential= polaris_key
          )
  return catalog
catalog  = connect_catalog(catalog_type)

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 13, Finished, Available, Finished)

In [5]:
db = "aemo"
catalog.create_namespace_if_not_exists(db)
catalog.list_namespaces()

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 14, Finished, Available, Finished)

[('aemo',)]

# Write

In [6]:
# @title
def get_table_files(db,table):
    table = catalog.load_table(db+'.'+table)
    zzz = table.scan(selected_fields=("file", )).to_arrow_batch_reader()
    table_files = duckdb.sql(f"select distinct file  from  zzz ").df()['file'].tolist()
    return table_files

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 15, Finished, Available, Finished)

In [7]:
# @title
def download(url,Path,total_files):
    if not os.path.exists(Path):
      os.makedirs(Path, exist_ok=True)
    result = urlopen(url).read().decode('utf-8')
    pattern = re.compile(r'[\w.]*.zip')
    filelist1 = pattern.findall(result)
    filelist_unique = dict.fromkeys(filelist1)
    filelist =sorted(filelist_unique, reverse=True)
    current =  [os.path.basename(x) for x in glob.glob(Path+'*.zip')]
    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload))[:total_files]
    print(str(len(files_to_upload)) + ' New File downloaded')
    if len(files_to_upload) != 0 :
      for x in files_to_upload:
           with requests.get(url+x, stream=True) as resp:
            if resp.ok:
              with open(f"{Path}{x}", "wb") as f:
               for chunk in resp.iter_content(chunk_size=4096):
                f.write(chunk)
    return "done"

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 16, Finished, Available, Finished)

In [8]:
# @title
def uncompress(x):
        unpack_archive(str(Source+x), str(Destination), 'zip')
def unzip(Source, Destination,Nbr_Files_to_Download):
    if not os.path.exists(Destination):
      os.makedirs(Destination, exist_ok=True)
    filelist=[os.path.basename(x) for x in glob.glob(Source+'*.zip')]
    ### checl the unzipped files already
    current = [os.path.basename(x) for x in glob.glob(Destination+'*.CSV')]
    current = [w.replace('.CSV','.zip') for w in current]
    #unzip only the delta
    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload))
    print(str(len(files_to_upload)) + ' New File uncompressed')
    if len(files_to_upload) != 0 :
      with multiprocessing.Pool() as pool:
       for _ in pool.imap_unordered(uncompress, files_to_upload, chunksize=1):
         pass
      return "done"
    else:
     return "nothing to see here"

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 17, Finished, Available, Finished)

In [9]:
# @title
def get_Path(Source,Destination):
 try:
  existing_files = get_table_files(db,Destination)
 except:
  existing_files = []
 print(len(existing_files))
 list_files_csv=[os.path.basename(x) for x in glob.glob(Source+'*.CSV')]
 list_files_json=[os.path.basename(x) for x in glob.glob(Source+'*.json')]
 filelist = list_files_csv + list_files_json

 files_to_upload = list(set(filelist) - set(existing_files))
 files_to_upload = list(dict.fromkeys(files_to_upload))
 files_to_upload_full_Path = [Source + i for i in files_to_upload]
 return files_to_upload_full_Path[:Nbr_Files_to_Download]

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 18, Finished, Available, Finished)

In [10]:
# @title
def clean_scada(files_to_upload_full_Path):
    raw =duckdb.sql(f"""from read_csv({files_to_upload_full_Path},
    Skip=1,header =0,all_varchar=1,
    columns={{
    'I': 'VARCHAR','UNIT': 'VARCHAR','XX': 'VARCHAR','VERSION': 'VARCHAR','SETTLEMENTDATE': 'VARCHAR','RUNNO': 'VARCHAR',
    'DUID': 'VARCHAR','INTERVENTION': 'VARCHAR','DISPATCHMODE': 'VARCHAR','AGCSTATUS': 'VARCHAR','INITIALMW': 'VARCHAR',
    'TOTALCLEARED': 'VARCHAR','RAMPDOWNRATE': 'VARCHAR','RAMPUPRATE': 'VARCHAR','LOWER5MIN': 'VARCHAR',
    'LOWER60SEC': 'VARCHAR','LOWER6SEC': 'VARCHAR','RAISE5MIN': 'VARCHAR','RAISE60SEC': 'VARCHAR',
    'RAISE6SEC': 'VARCHAR','MARGINAL5MINVALUE': 'VARCHAR','MARGINAL60SECVALUE': 'VARCHAR',
    'MARGINAL6SECVALUE': 'VARCHAR','MARGINALVALUE': 'VARCHAR','VIOLATION5MINDEGREE': 'VARCHAR',
    'VIOLATION60SECDEGREE': 'VARCHAR','VIOLATION6SECDEGREE': 'VARCHAR','VIOLATIONDEGREE': 'VARCHAR',
    'LOWERREG': 'VARCHAR','RAISEREG': 'VARCHAR','AVAILABILITY': 'VARCHAR','RAISE6SECFLAGS': 'VARCHAR',
    'RAISE60SECFLAGS': 'VARCHAR','RAISE5MINFLAGS': 'VARCHAR','RAISEREGFLAGS': 'VARCHAR',
    'LOWER6SECFLAGS': 'VARCHAR','LOWER60SECFLAGS': 'VARCHAR','LOWER5MINFLAGS': 'VARCHAR',
    'LOWERREGFLAGS': 'VARCHAR','RAISEREGAVAILABILITY': 'VARCHAR','RAISEREGENABLEMENTMAX': 'VARCHAR',
    'RAISEREGENABLEMENTMIN': 'VARCHAR','LOWERREGAVAILABILITY': 'VARCHAR','LOWERREGENABLEMENTMAX': 'VARCHAR',
    'LOWERREGENABLEMENTMIN': 'VARCHAR','RAISE6SECACTUALAVAILABILITY': 'VARCHAR',
    'RAISE60SECACTUALAVAILABILITY': 'VARCHAR','RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
    'RAISEREGACTUALAVAILABILITY': 'VARCHAR','LOWER6SECACTUALAVAILABILITY': 'VARCHAR',
    'LOWER60SECACTUALAVAILABILITY': 'VARCHAR','LOWER5MINACTUALAVAILABILITY': 'VARCHAR','LOWERREGACTUALAVAILABILITY': 'VARCHAR'
    }},
    filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
    where I='D' and UNIT ='DUNIT' AND VERSION = 3                  """)

    final_scada=duckdb.sql("""
    select
    UNIT,
    DUID,
    cast(columns(*exclude(DUID,UNIT,SETTLEMENTDATE,I,XX,filename)) as double),
    parse_filename(filename) as file,
    0 as PRIORITY ,
    cast (SETTLEMENTDATE as TIMESTAMPTZ) as SETTLEMENTDATE,
    cast(SETTLEMENTDATE as date) as date,
	  isoyear (cast (SETTLEMENTDATE as timestamp)) as YEAR
    from raw  """)
    return final_scada.arrow()

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 19, Finished, Available, Finished)

In [11]:
# @title
def clean_price(files_to_upload_full_Path):
  raw =duckdb.sql(f"""from read_csv({files_to_upload_full_Path},
  Skip=1,header =0,all_varchar=1,
  columns={{
  'I': 'VARCHAR','UNIT': 'VARCHAR','XX': 'VARCHAR','VERSION': 'VARCHAR','SETTLEMENTDATE': 'VARCHAR','RUNNO': 'VARCHAR',
  'REGIONID': 'VARCHAR','INTERVENTION': 'VARCHAR','RRP': 'VARCHAR','EEP': 'VARCHAR','ROP': 'VARCHAR','APCFLAG': 'VARCHAR',
  'MARKETSUSPENDEDFLAG': 'VARCHAR','TOTALDEMAND': 'VARCHAR','DEMANDFORECAST': 'VARCHAR','DISPATCHABLEGENERATION': 'VARCHAR',
  'DISPATCHABLELOAD': 'VARCHAR','NETINTERCHANGE': 'VARCHAR','EXCESSGENERATION': 'VARCHAR','LOWER5MINDISPATCH': 'VARCHAR',
  'LOWER5MINIMPORT': 'VARCHAR','LOWER5MINLOCALDISPATCH': 'VARCHAR','LOWER5MINLOCALPRICE': 'VARCHAR','LOWER5MINLOCALREQ': 'VARCHAR',
  'LOWER5MINPRICE': 'VARCHAR','LOWER5MINREQ': 'VARCHAR','LOWER5MINSUPPLYPRICE': 'VARCHAR','LOWER60SECDISPATCH': 'VARCHAR','LOWER60SECIMPORT': 'VARCHAR',
  'LOWER60SECLOCALDISPATCH': 'VARCHAR','LOWER60SECLOCALPRICE': 'VARCHAR','LOWER60SECLOCALREQ': 'VARCHAR','LOWER60SECPRICE': 'VARCHAR',
  'LOWER60SECREQ': 'VARCHAR','LOWER60SECSUPPLYPRICE': 'VARCHAR','LOWER6SECDISPATCH': 'VARCHAR','LOWER6SECIMPORT': 'VARCHAR',
  'LOWER6SECLOCALDISPATCH': 'VARCHAR','LOWER6SECLOCALPRICE': 'VARCHAR','LOWER6SECLOCALREQ': 'VARCHAR','LOWER6SECPRICE': 'VARCHAR',
  'LOWER6SECREQ': 'VARCHAR','LOWER6SECSUPPLYPRICE': 'VARCHAR','RAISE5MINDISPATCH': 'VARCHAR','RAISE5MINIMPORT': 'VARCHAR',
  'RAISE5MINLOCALDISPATCH': 'VARCHAR','RAISE5MINLOCALPRICE': 'VARCHAR','RAISE5MINLOCALREQ': 'VARCHAR','RAISE5MINPRICE': 'VARCHAR',
  'RAISE5MINREQ': 'VARCHAR','RAISE5MINSUPPLYPRICE': 'VARCHAR','RAISE60SECDISPATCH': 'VARCHAR','RAISE60SECIMPORT': 'VARCHAR',
  'RAISE60SECLOCALDISPATCH': 'VARCHAR','RAISE60SECLOCALPRICE': 'VARCHAR','RAISE60SECLOCALREQ': 'VARCHAR','RAISE60SECPRICE': 'VARCHAR',
  'RAISE60SECREQ': 'VARCHAR','RAISE60SECSUPPLYPRICE': 'VARCHAR','RAISE6SECDISPATCH': 'VARCHAR','RAISE6SECIMPORT': 'VARCHAR',
  'RAISE6SECLOCALDISPATCH': 'VARCHAR','RAISE6SECLOCALPRICE': 'VARCHAR','RAISE6SECLOCALREQ': 'VARCHAR','RAISE6SECPRICE': 'VARCHAR',
  'RAISE6SECREQ': 'VARCHAR','RAISE6SECSUPPLYPRICE': 'VARCHAR','AGGREGATEDISPATCHERROR': 'VARCHAR','AVAILABLEGENERATION': 'VARCHAR',
  'AVAILABLELOAD': 'VARCHAR','INITIALSUPPLY': 'VARCHAR','CLEAREDSUPPLY': 'VARCHAR','LOWERREGIMPORT': 'VARCHAR','LOWERREGLOCALDISPATCH': 'VARCHAR',
  'LOWERREGLOCALREQ': 'VARCHAR','LOWERREGREQ': 'VARCHAR','RAISEREGIMPORT': 'VARCHAR','RAISEREGLOCALDISPATCH': 'VARCHAR','RAISEREGLOCALREQ': 'VARCHAR',
  'RAISEREGREQ': 'VARCHAR','RAISE5MINLOCALVIOLATION': 'VARCHAR','RAISEREGLOCALVIOLATION': 'VARCHAR','RAISE60SECLOCALVIOLATION': 'VARCHAR',
  'RAISE6SECLOCALVIOLATION': 'VARCHAR','LOWER5MINLOCALVIOLATION': 'VARCHAR','LOWERREGLOCALVIOLATION': 'VARCHAR','LOWER60SECLOCALVIOLATION': 'VARCHAR',
  'LOWER6SECLOCALVIOLATION': 'VARCHAR','RAISE5MINVIOLATION': 'VARCHAR','RAISEREGVIOLATION': 'VARCHAR','RAISE60SECVIOLATION': 'VARCHAR',
  'RAISE6SECVIOLATION': 'VARCHAR','LOWER5MINVIOLATION': 'VARCHAR','LOWERREGVIOLATION': 'VARCHAR','LOWER60SECVIOLATION': 'VARCHAR',
  'LOWER6SECVIOLATION': 'VARCHAR','RAISE6SECRRP': 'VARCHAR','RAISE6SECROP': 'VARCHAR','RAISE6SECAPCFLAG': 'VARCHAR','RAISE60SECRRP': 'VARCHAR',
  'RAISE60SECROP': 'VARCHAR','RAISE60SECAPCFLAG': 'VARCHAR','RAISE5MINRRP': 'VARCHAR','RAISE5MINROP': 'VARCHAR','RAISE5MINAPCFLAG': 'VARCHAR',
  'RAISEREGRRP': 'VARCHAR','RAISEREGROP': 'VARCHAR','RAISEREGAPCFLAG': 'VARCHAR','LOWER6SECRRP': 'VARCHAR','LOWER6SECROP': 'VARCHAR',
  'LOWER6SECAPCFLAG': 'VARCHAR','LOWER60SECRRP': 'VARCHAR','LOWER60SECROP': 'VARCHAR','LOWER60SECAPCFLAG': 'VARCHAR','LOWER5MINRRP': 'VARCHAR',
  'LOWER5MINROP': 'VARCHAR','LOWER5MINAPCFLAG': 'VARCHAR','LOWERREGRRP': 'VARCHAR','LOWERREGROP': 'VARCHAR','LOWERREGAPCFLAG': 'VARCHAR',
  'RAISE6SECACTUALAVAILABILITY': 'VARCHAR','RAISE60SECACTUALAVAILABILITY': 'VARCHAR','RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
  'RAISEREGACTUALAVAILABILITY': 'VARCHAR','LOWER6SECACTUALAVAILABILITY': 'VARCHAR','LOWER60SECACTUALAVAILABILITY': 'VARCHAR',
  'LOWER5MINACTUALAVAILABILITY': 'VARCHAR','LOWERREGACTUALAVAILABILITY': 'VARCHAR','LORSURPLUS': 'VARCHAR','LRCSURPLUS': 'VARCHAR',
  }},
  filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
  where I='D' and UNIT ='DREGION' AND VERSION = 3
                  """)
  final_price=duckdb.sql("""
    select
    UNIT,
    REGIONID,
    cast(columns(*exclude(REGIONID,UNIT,SETTLEMENTDATE,I,XX,filename)) as double),
    parse_filename(filename) as file,
    0 as PRIORITY ,
    cast (SETTLEMENTDATE as TIMESTAMPTZ) as SETTLEMENTDATE,
    cast(SETTLEMENTDATE as date) as date,
	  isoyear (cast (SETTLEMENTDATE as timestamp)) as YEAR
    from raw  """)
  return final_price.arrow()

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 20, Finished, Available, Finished)

**<mark>calendar</mark>**

In [12]:
# @title
tbl = db+"."+"calendar"
if not catalog.table_exists(tbl):
  df=duckdb.sql(""" SELECT cast(unnest(generate_series(cast ('2018-04-01' as date), cast('2024-12-31' as date), interval 1 day)) as date) as date,
            EXTRACT(year from date) as year,
            EXTRACT(month from date) as month
            """).arrow()
  catalog.create_table_if_not_exists(tbl,schema=df.schema)
  catalog.load_table(tbl).overwrite(df)
  print('calendar created')
else:
    print("table exist already")

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 21, Finished, Available, Finished)

table exist already


**<mark>mstdatetime</mark>**

In [13]:
# @title
tbl = db+"."+"mstdatetime"
if not catalog.table_exists(tbl):
  df=duckdb.sql(""" SELECT cast(unnest(generate_series(cast ('2018-04-01' as date), cast('2024-12-31' as date), interval 5 minute)) as TIMESTAMPTZ) as SETTLEMENTDATE,
           strftime(SETTLEMENTDATE, '%I:%M:%S %p') as time,
           cast(SETTLEMENTDATE as date ) as date,
           EXTRACT(year from date) as year,
           EXTRACT(month from date) as month
           """).arrow()
  catalog.create_table_if_not_exists(tbl,schema=df.schema)
  catalog.load_table(tbl).overwrite(df)
  print('mstdatetime created')
else:
    print("table exist already")

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 22, Finished, Available, Finished)

table exist already


**<mark>DUID</mark>**

In [14]:
# @title
DUID_Path = "/lakehouse/default/Files/0_Source/Dimensions/DUID/"
import pathlib
pathlib.Path(DUID_Path).mkdir(parents=True, exist_ok=True)
url = "https://www.aemo.com.au/-/media/Files/Electricity/NEM/Participant_Information/NEM-Registration-and-Exemption-List.xls"
s = requests.Session()
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
r = s.get(url,headers=headers)
r.content
output = open(DUID_Path+"NEM-Registration-and-Exemption-List.xls", 'wb')
output.write(r.content)
output.close()
duckdb.sql(f"""
INSTALL spatial;
LOAD spatial;
create or replace table DUID as
SELECT Region,DUID,first("Fuel Source - Descriptor") as FuelSourceDescriptor,first(Participant) as Participant
FROM st_read('{DUID_Path}NEM-Registration-and-Exemption-List.xls', layer = 'PU and Scheduled Loads',open_options = ['HEADERS=FORCE'])
group by all
""")

import requests
dls = "https://data.wa.aemo.com.au/datafiles/post-facilities/facilities.csv"
resp = requests.get(dls)
output = open(DUID_Path+"facilities_WA.csv", 'wb')
output.write(resp.content)
output.close()

duckdb.sql(f""" select 'WA1' as Region  , "Facility Code" as DUID ,"Participant Name" as Participant from read_csv_auto('{DUID_Path}facilities_WA.csv')""").to_view('x')

duckdb.sql("""select x.Region,x.DUID, TECHNOLOGY as FuelSourceDescriptor,Participant from x
left join (select * FROM read_csv_auto('https://github.com/djouallah/aemo_fabric/raw/main/WA_ENERGY.csv',header=1)) as z
on x.duid=z.duid """).to_view('DUID_WA')

duckdb.sql("""
create or replace table states(RegionID varchar, States varchar) ;
insert into states values
  ('WA1' , 'Western Australia') ,
  ('QLD1' , 'Queensland')  ,
  ('NSW1' , 'New South Walles')  ,
  ('TAS1' , 'Tasmania')  ,
  ('SA1' , 'South Australia')  ,
  ('VIC1' , 'Victoria')
   """)

df =duckdb.sql(f""" with xx as (select * from DUID union BY NAME select * from DUID_WA)
              select States,trim(DUID) as DUID,min(Region) as Region, min(FuelSourceDescriptor) as FuelSourceDescriptor,
              min(Participant) as Participant
              from xx 
              JOIN states on xx.Region = states.RegionID
              where len(trim(DUID)) > 3
              group by all
              """).arrow()
catalog.create_table_if_not_exists(db+".duid",schema=df.schema)
catalog.load_table(db+".duid").overwrite(df)
print('duid updated')

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 23, Finished, Available, Finished)

/nfs4/pyenv-7ef9544c-8c88-4e53-a025-a245d80d9eff/lib/python3.11/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Table.identifier property is deprecated. Please use Table.name() function instead.
  _deprecation_warning(deprecation_notice(deprecated_in, removed_in, help_message))
/nfs4/pyenv-7ef9544c-8c88-4e53-a025-a245d80d9eff/lib/python3.11/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Support for parsing catalog level identifier in Catalog identifiers is deprecated. Please refer to the table using only its namespace and its table name.
  _deprecation_warning(deprecation_notice(deprecated_in, removed_in, help_message))
/nfs4/pyenv-7ef9544c-8c88-4e53-a025-a245d80d9eff/lib/python3.11/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Table.identifier property is deprecated. Please use Table.

**<mark>Fact Tables</mark>**

In [15]:
Nbr_Files_to_Download = 60
Source                = "/lakehouse/default/Files/0_Source/aemo/"
Destination           = "/lakehouse/default/Files/1_Transform/CSV/Daily_Reports/"
download("https://nemweb.com.au/Reports/Current/Daily_Reports/", Source, Nbr_Files_to_Download)
unzip(Source, Destination,Nbr_Files_to_Download)
for tbl in ['scada', 'price']:
        files_to_upload_full_Path = get_Path(Destination, tbl)
        if len(files_to_upload_full_Path) > 0:
            df = eval(f"clean_{tbl}(files_to_upload_full_Path)")
            catalog.create_table_if_not_exists(f'{db}.{tbl}', schema=df.schema)
            catalog.load_table(f'{db}.{tbl}').append(df)
            print(f'{tbl} updated')
        else:
            print(f'{tbl} loaded already')

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 24, Finished, Available, Finished)

58 New File downloaded
58 New File uncompressed
2


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

scada updated
2


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

/nfs4/pyenv-7ef9544c-8c88-4e53-a025-a245d80d9eff/lib/python3.11/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Table.identifier property is deprecated. Please use Table.name() function instead.
  _deprecation_warning(deprecation_notice(deprecated_in, removed_in, help_message))
/nfs4/pyenv-7ef9544c-8c88-4e53-a025-a245d80d9eff/lib/python3.11/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Support for parsing catalog level identifier in Catalog identifiers is deprecated. Please refer to the table using only its namespace and its table name.
  _deprecation_warning(deprecation_notice(deprecated_in, removed_in, help_message))
/nfs4/pyenv-7ef9544c-8c88-4e53-a025-a245d80d9eff/lib/python3.11/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Table.identifier property is deprecated. Please use Table.

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 31, Finished, Available, Finished)

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 32, Finished, Available, Finished)

# Read

In [16]:
daft.catalog.register_python_catalog(catalog)
scada = daft.read_table("aemo.scada")
duid = daft.read_table("aemo.duid")


StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 25, Finished, Available, Finished)

/nfs4/pyenv-7ef9544c-8c88-4e53-a025-a245d80d9eff/lib/python3.11/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Table.identifier property is deprecated. Please use Table.name() function instead.
  _deprecation_warning(deprecation_notice(deprecated_in, removed_in, help_message))
/nfs4/pyenv-7ef9544c-8c88-4e53-a025-a245d80d9eff/lib/python3.11/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Table.identifier property is deprecated. Please use Table.name() function instead.
  _deprecation_warning(deprecation_notice(deprecated_in, removed_in, help_message))


In [17]:
daft.sql("""
select States, sum(INITIALMW)/12 as mwh from scada
join duid on scada.DUID = duid.DUID
group by States
order by mwh
 """).show()

StatesUtf8,mwhFloat64
Tasmania,1357151.463801644
South Australia,1502995.6279907525
Victoria,6706161.672593803
New South Walles,7432508.6723924475
Queensland,9060048.057647727


In [18]:
%%sql
select States, sum(INITIALMW)/12 as mwh from scada
join duid on scada.DUID = duid.DUID
group by States
order by mwh

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 27, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 2 fields>

# Check Catalog Database

In [19]:
duckdb.sql(f"""
INSTALL sqlite;
LOAD sqlite;
ATTACH '{db_path}' (TYPE SQLITE);
""")

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 28, Finished, Available, Finished)

In [20]:
display(duckdb.sql(""" use pyiceberg_catalog; from iceberg_tables  """).df())

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 29, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9d9b15ac-a890-4649-87a4-4ed3b2c09ff0)

In [21]:
display(duckdb.sql(""" from iceberg_namespace_properties  """).df())

StatementMeta(, 59b3d2a9-f10b-4535-ac44-0e51344c1249, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2c9d5c09-ba6b-4522-b9ac-f30cc7d5de52)